In [1]:
import datetime
import numpy as np
from random import randint
import os
# from sklearn import cross_validation
from sklearn.model_selection import cross_validate
# from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import pandas as pd
import math
from sklearn import preprocessing
from sklearn import metrics
# from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
# from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import learning_curve, GridSearchCV

# from sklearn.cross_validation import train_test_split,StratifiedShuffleSplit,StratifiedKFold
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,StratifiedKFold
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
import time
from sklearn.externals import joblib
from sklearn.metrics import classification_report
import pickle

print("Import ok")

Import ok


In [2]:
def validate_predictions(jobs_test, predictions):
    mean_real_pow = jobs_test['real_pow'].mean()
    print("the mean is {}".format(mean_real_pow))

    eq_num = (jobs_test['real_pow'] - predictions)**2
    eq_den = (jobs_test['real_pow'] - mean_real_pow)**2
    N = eq_num.shape[0]

    r_2 = 1 - (eq_num.sum() / eq_den.sum())
    print("r_2 is {}".format(r_2))

    rmse = math.sqrt(eq_num.sum() / N)
    print("RMSE is {}".format(rmse))
    print("NRMSE is {}".format(rmse/mean_real_pow))

## Reading the datasets

CSV file: contains the data set containing jobs and their real power consumption, info that were used to train the ML models.

This data set contains info for each job provided by users at submission time and its "approximated" power consumption; approximated because we could only be sure of the power  consumption of jobs that used whole nodes -- the consumption of jobs that shared a node with other ones were computed as described before (hence the "approximate" -- the estimate is pretty accurate, 4%-%6 error as you can see in [3]).

In [3]:
node_idle_power = 45
plot_on = 0

datadir = "/datasets/eurora_data/"
infile = datadir + "jobs_powers_jobs_approx.csv"
infile = datadir + "jobs_powers_jobs_approx_computedWith_ncoreActive_profile.csv"
# version with whole system info
infile = datadir + "jobs_powers_jobs_approx_computedWith_ncoreActive_profile_wholeSystemInfo.csv"
    
# read dataset
mydata = pd.read_csv(infile) # Read a comma-separated values (csv) file into DataFrame
mydata = mydata.dropna() # remove missing values

target = mydata.job_tot
#target = mydata.core_avg_load
max_real_dur = max(mydata.real_dur)
max_req_dur = max(mydata.req_dur)


mydata.keys()

Index(['user', 'queue', 'req_dur', 'real_dur', 'nnodes_req', 'ncore_req',
       'mem_req', 'ngpus_req', 'nmics_req', 'mixed_nodes', 'n2_1', 'n3_1',
       'node_type', 'core_avg_load', 'core_avg_cpi', 'core_avg_temp',
       'core_avg_ips', 'core_avg_rfreq', 'avg_pow_cpu', 'avg_pow_dram',
       'avg_pow_pkg', 'avg_perCpu_perJobUnit_pow', 'njobs_system',
       'ncores_system', 'load_system', 'job_tot'],
      dtype='object')

In [4]:
# mydata entries

mydata.shape

(70951, 26)

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


mydata.head(10)

,user,queue,req_dur,real_dur,nnodes_req,ncore_req,mem_req,ngpus_req,nmics_req,mixed_nodes,n2_1,n3_1,node_type,core_avg_load,core_avg_cpi,core_avg_temp,core_avg_ips,core_avg_rfreq,avg_pow_cpu,avg_pow_dram,avg_pow_pkg,avg_perCpu_perJobUnit_pow,njobs_system,ncores_system,load_system,job_tot
0,framondo,parallel,3600.0,202.0,1,16,14,0,0,0,1,0,2_1_ghz,92.356961,1.153844,37.481250,1.689856e+09,2047.868633,38.052675,10.133666,58.973523,46.107190,2,32,0.031250,93.214379
1,vcalzia0,parallel,14400.0,14439.0,6,96,84,0,0,0,6,0,2_1_ghz,85.735984,0.595046,39.119758,3.113065e+09,2113.203376,39.832029,6.318053,57.049498,40.050211,7,112,0.109375,31.202511
2,alupi000,parallel,14400.0,14476.0,1,16,14,0,0,0,1,0,2_1_ghz,99.906312,0.442679,38.748731,4.748928e+09,2099.594705,39.175164,6.544391,56.357617,39.902008,8,128,0.125000,80.804016
3,a07smr01,R472836,21600.0,13794.0,1,1,2,0,0,0,0,1,3_1_ghz,6.529423,1.836259,39.852095,2.695428e+08,1789.441123,11.786934,3.195254,28.217508,8.412762,9,129,0.125977,1.655414
4,a07smr01,R472836,21600.0,1392.0,11,176,154,0,0,0,0,11,3_1_ghz,71.051961,1.374327,58.307690,2.637564e+09,2863.346384,71.385445,6.627938,90.371316,74.043080,20,305,0.297852,102.852167
5,a07smr01,R472836,21600.0,3373.0,11,176,154,0,0,0,0,11,3_1_ghz,25.596916,2.115308,45.640307,8.923187e+08,1783.515706,27.944428,4.249580,45.320450,26.791144,24,316,0.308594,37.741317
6,a07smr01,R472836,21600.0,12094.0,1,8,2,0,0,0,0,1,3_1_ghz,7.356625,1.759659,40.151771,3.060262e+08,1865.610280,12.962087,3.304669,29.434060,9.738729,21,313,0.305664,13.580148
7,a07smr01,R472836,21600.0,19362.0,1,1,2,0,0,0,0,1,3_1_ghz,5.076829,1.829221,39.236198,1.963118e+08,1627.234464,9.434283,3.000649,25.814916,5.815565,22,314,0.306641,1.468570
8,a07smr01,R472836,21600.0,18626.0,1,1,2,0,0,0,0,1,3_1_ghz,5.117825,1.800260,39.272029,2.020449e+08,1640.247077,9.580945,2.981124,25.953462,5.934586,24,316,0.308594,1.445838
9,a07smr01,R472836,21600.0,12098.0,1,1,2,0,0,0,0,1,3_1_ghz,7.357419,1.759741,40.152019,3.059760e+08,1865.616793,12.962529,3.304918,29.434546,9.739465,24,316,0.308594,1.698074


In [6]:
# drop unused (or not usable) info
data = mydata.drop(["real_dur"],axis=1)
data = data.drop(["core_avg_load"],axis=1)
data = data.drop(["core_avg_rfreq"],axis=1)
data = data.drop(["core_avg_cpi"],axis=1)
data = data.drop(["core_avg_temp"],axis=1)
data = data.drop(["core_avg_ips"],axis=1)
data = data.drop(["avg_pow_cpu"],axis=1)
data = data.drop(["avg_pow_dram"],axis=1)
data = data.drop(["avg_pow_pkg"],axis=1)
data = data.drop(["avg_perCpu_perJobUnit_pow"],axis=1)
data = data.drop(["job_tot"],axis=1)

data.keys()

Index(['user', 'queue', 'req_dur', 'nnodes_req', 'ncore_req', 'mem_req', 'ngpus_req', 'nmics_req', 'mixed_nodes', 'n2_1', 'n3_1', 'node_type', 'njobs_system', 'ncores_system', 'load_system'], dtype='object')

In [7]:
data.head(5)

,user,queue,req_dur,nnodes_req,ncore_req,mem_req,ngpus_req,nmics_req,mixed_nodes,n2_1,n3_1,node_type,njobs_system,ncores_system,load_system
0,framondo,parallel,3600.0,1,16,14,0,0,0,1,0,2_1_ghz,2,32,0.031250
1,vcalzia0,parallel,14400.0,6,96,84,0,0,0,6,0,2_1_ghz,7,112,0.109375
2,alupi000,parallel,14400.0,1,16,14,0,0,0,1,0,2_1_ghz,8,128,0.125000
3,a07smr01,R472836,21600.0,1,1,2,0,0,0,0,1,3_1_ghz,9,129,0.125977
4,a07smr01,R472836,21600.0,11,176,154,0,0,0,0,11,3_1_ghz,20,305,0.297852


In [8]:
# normalize 
#data['req_dur'] = (data['req_dur'] - data['req_dur'].mean()) / (data['req_dur'].max() - data['req_dur'].min())
#data['nnodes_req'] = (data['nnodes_req'] - data['nnodes_req'].mean()) / (data['nnodes_req'].max() - data['nnodes_req'].min())
#data['ncore_req'] = (data['ncore_req'] - data['ncore_req'].mean()) / (data['ncore_req'].max() - data['ncore_req'].min())
#data['mem_req'] = (data['mem_req'] - data['mem_req'].mean()) / (data['mem_req'].max() - data['mem_req'].min())
#data['ngpus_req'] = (data['ngpus_req'] - data['ngpus_req'].mean()) / (data['ngpus_req'].max() - data['ngpus_req'].min())
#data['nmics_req'] = (data['nmics_req'] - data['nmics_req'].mean()) / (data['nmics_req'].max() - data['nmics_req'].min())

#norm_mean = target.mean()
#norm_max = target.max()
#norm_min = target.min()
#target = (target - target.mean()) / target.max() - target.min()

# problem: when new categorical values arrive (user,queues..) the ML model still wants
# n_train_time number of features --> this is a horrible workaround which relies
# on the fact that the test will use a subset of the jobs in the train set
qs = []
for q in data['queue']:
    if not q in qs:
        qs.append(q)
us = []
for u in data['user']:
    if not u in us:
        us.append(u)
nts = []
for nt in data['node_type']:
    if not nt in nts:
        nts.append(nt)

# convert categorical data
dms = pd.get_dummies(data['node_type'])
data = pd.concat([data, dms], axis=1)    
data = data.drop(['node_type'], axis=1)

dms = pd.get_dummies(data['queue'])
data = pd.concat([data, dms], axis=1)    
data = data.drop(['queue'], axis=1)

dms = pd.get_dummies(data['user'])
data = pd.concat([data, dms], axis=1)    
data = data.drop(['user'], axis=1)

# delete rows  with missing values
data = data.dropna()
target = target.dropna()

features_list = list(data.columns)
# print(features_list)

# convert panda dataframe into numpy arrays
data_np = np.array(data)
target_np = np.array(target)

# scale (obtain data with zero mean and unit variance)
#data_scaled = preprocessing.scale(data_np)
#target_scaled = preprocessing.scale(target_np)
data_scaled = data_np
target_scaled = target_np

data.iloc[:,5:].head(5)


,nmics_req,mixed_nodes,n2_1,n3_1,njobs_system,ncores_system,load_system,2_1_ghz,3_1_ghz,R472836,R498534,R498535,R498536,R498537,R505112,R509757,R544947,R563598,R563599,R563600,R563601,R563602,R563603,R565862,R588271,R588272,R588273,R588274,R588275,R588276,R588277,R588278,R588279,R588280,R588281,R588283,R588286,R588287,R588288,R588289,R588290,R588291,R588293,R588294,R588295,R588296,R588297,R588298,R588607,R588642,R628233,R628989,R628990,R628993,R628994,R628996,R628999,R629000,R629001,R629002,R635731,debug,meteopar,np_longpar,p_devel,parallel,route,system,visual,a07smr01,a08tra59,a08tra60,a08tra61,a08tra62,a08tra63,a08tra64,aalbinat,aambrose,abiancar,acavalli,aemerson,afanfari,afarrott,aferrar2,ageorgou,agrottes,alorenzo,alupi000,amagistr,amarini0,amelchio,amessina,amotta00,aolarini,apalma00,apietro1,aprestig,asaetti0,aserva00,aspitale,avargiu0,bcatalan,bziosi00,ccaddeo0,ccavazzo,ccostabi,cmorgill,cpadrin0,cplatani,dbochicc,dburatto,dcagnoni,dcrivell,dmarson0,dmeggiol,dmelazzi,ebodo000,ecoccia0,elaurini,emacerat,emarcucc,emossini,epapaleo,epascolo,estalio0,faffinit,fbeneven,fcerutti,fferre00,ffrater1,fgabas00,fmercuri,fnegreir,fnicolus,fpetracc,fraimond,framondo,fstellat,ftramont,fzonta00,gamati01,gbarcaro,gbertain,gcalo000,gchillem,gdadamo0,gdavino0,gfatigat,ggatti00,ggrazios,glasala0,glonghi0,gmagno00,gquarant,gtiana00,ibaccare,idaidone,ispisso0,jfranz00,jjuarezj,jolmosas,jsgrigna,kkwapien,ldegioia,lfalciol,lloprest,lmucciol,lorian00,lraimond,lsementa,manselmi,marcased,mbenfatt,mborelli,mcaccial,mcalabre,mceotto0,mcerini0,mcremone,mdistefa,mfarnesi,mgrande0,mippolit,mmaccion,mmapelli,mmorgott,mnini000,mpalummo,mpividor,mrutigli,msaba000,mschroec,nspalla1,ntasinat,oroscion,pcacciot,pciottol,pfrascon,pgiannoz,planucar,pleoni00,pposocco,psilvest,pstipa00,ralfieri,rbizzott,rborrell,rcini000,rgaleazz,rjanusze,rmucci00,rnifosi0,rponzini,rvacondi,scalza00,sdecherc,spezzott,sraniolo,sravelli,sschifan,szaghi00,szia0000,tbusatta,tgiorgin,vcalzia0,vdalcol0,vmarino0,vruggie1,wxie0000
0,0,0,1,0,2,32,0.031250,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,6,0,7,112,0.109375,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,8,128,0.125000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,9,129,0.125977,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,11,20,305,0.297852,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [9]:
# create test and training set
train_data, test_data, train_target, test_target = train_test_split(data_scaled, target_scaled, test_size = 0.2, random_state = 42)

print(train_data.shape[0])
print(test_data.shape[0])

regr = RandomForestRegressor()
regr.fit(train_data,train_target)

print("[RandomForestRegressor] Train data Accuracy: %0.2f " % regr.score(train_data, train_target))


print("[RandomForestRegressor] Test data Accuracy: %0.2f " % regr.score(test_data, test_target))

56760
14191


/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[RandomForestRegressor] Train data Accuracy: 0.94 
[RandomForestRegressor] Test data Accuracy: 0.68 


In [10]:
print("[RandomForestRegressor] Whole data Accuracy: %0.2f " % regr.score(np.concatenate([train_data, test_data]), np.concatenate([train_target, test_target])))


[RandomForestRegressor] Whole data Accuracy: 0.88 


In [11]:
predictions = regr.predict(test_data)

In [12]:
def validate_predictions(test_target, predictions):
    mean_real_pow = test_target.mean()
    print("the mean is {}".format(mean_real_pow))

    eq_num = (test_target - predictions)**2
    eq_den = (test_target - mean_real_pow)**2
    N = eq_num.shape[0]

    r_2 = 1 - (eq_num.sum() / eq_den.sum())
    print("r_2 is {}".format(r_2))

    rmse = math.sqrt(eq_num.sum() / N)
    print("RMSE is {}".format(rmse))
    print("NRMSE is {}".format(rmse/mean_real_pow))

In [13]:
validate_predictions(test_target, predictions)

the mean is 43.10532226310283
r_2 is 0.6753633432538643
RMSE is 53.98830506352249
NRMSE is 1.2524742242731182


In [22]:
#important_features_dict ={}
#
#for x,i in enumerate(regr.feature_importances_):
#    important_features_dict[x] = i
#    
#important_features_list = sorted(important_features_dict, key=important_features_dict.get, reverse=True)
#
## print("Most important features: %s" %important_features_list)
#
#for x,i in enumerate(important_features_list):
#    print(str(i) + "\t" + features_list[i])

## Test a prediction

In [23]:
test_data[0:1]

for i,feature in enumerate(features_list):
    print(feature + "\t\t" + str(test_data[0:1][0][i]))

req_dur		1800.0
nnodes_req		0.0
ncore_req		0.0
mem_req		1.0
ngpus_req		0.0
nmics_req		0.0
mixed_nodes		0.0
n2_1		0.0
n3_1		1.0
njobs_system		121.0
ncores_system		709.0
load_system		0.6923828125
2_1_ghz		0.0
3_1_ghz		1.0
R472836		0.0
R498534		0.0
R498535		0.0
R498536		0.0
R498537		0.0
R505112		0.0
R509757		0.0
R544947		0.0
R563598		0.0
R563599		0.0
R563600		0.0
R563601		0.0
R563602		0.0
R563603		0.0
R565862		0.0
R588271		0.0
R588272		0.0
R588273		0.0
R588274		0.0
R588275		0.0
R588276		0.0
R588277		0.0
R588278		0.0
R588279		0.0
R588280		0.0
R588281		0.0
R588283		0.0
R588286		0.0
R588287		0.0
R588288		0.0
R588289		0.0
R588290		0.0
R588291		0.0
R588293		0.0
R588294		0.0
R588295		0.0
R588296		0.0
R588297		0.0
R588298		0.0
R588607		0.0
R588642		0.0
R628233		0.0
R628989		0.0
R628990		0.0
R628993		0.0
R628994		0.0
R628996		0.0
R628999		0.0
R629000		0.0
R629001		0.0
R629002		0.0
R635731		0.0
debug		1.0
meteopar		0.0
np_longpar		0.0
p_devel		0.0
parallel		0.0
route		0.0
system		0.0
visual		0.0
a

In [24]:
print(regr.predict(test_data[0:1]))

[15.4425132]


# Pickle data info

The information about which jobs are active on the system at every moment and how many cores are active, were stored in two pickle files (nCoresActive_wholeSystem_profile.pickle and nJobs_wholeSystem_profile.pickle).

In [2]:
datadir = "/datasets/eurora_data/"
# read whole system info (they can be used as an input); obviously in real application these
# info should be colletected at run time
ncores_system_profile_file = datadir + "nCorAct_wSys_prof.pickle"
with open(ncores_system_profile_file, 'rb') as handle:
    system_activeCores = pickle.load(handle, encoding='bytes')
    

njobs_system_profile_file =  datadir + "njobs_wholeSystem_profiles.pickle"
with open(njobs_system_profile_file, 'rb') as handle:
    system_nJobs = pickle.load(handle, encoding='bytes')

In [3]:
pickle_len = len(system_activeCores) # 132416
first_key = list(system_activeCores.keys())[0]
last_key = list(system_activeCores.keys())[len(system_activeCores)-1]

print("There are {} entries, from {} to {}".format(pickle_len, first_key, last_key))

There are 132416 entries, from 2014-03-31 09:10:54 to 2014-05-01 15:06:20


In [4]:
print("There are {} active cores, at {}".format(list(system_activeCores.values())[0], list(system_activeCores.keys())[0]))

There are 16 active cores, at 2014-03-31 09:10:54


In [28]:
#len(system_nJobs) # 142849

In [29]:
# count the occurrences of the same date

# dict = {}
# counter = 0

# for key,value in system_activeCores.items():
#     if (system_nJobs[key] != None):
#         print(str(value) + " : " + str(system_nJobs[key]))
#         counter +=1
#
# print(counter) # 132416

In [5]:
pickle_len = len(system_nJobs) # 132416
first_key = list(system_nJobs.keys())[0]
second_key = list(system_nJobs.keys())[1]
third_key = list(system_nJobs.keys())[2]
last_key = list(system_nJobs.keys())[len(system_nJobs)-1]

print("There are {} entries, from {} to {}".format(pickle_len, first_key, last_key))

There are 142849 entries, from 2014-03-31 00:30:02 to 2014-05-09 10:40:57


In [6]:
print("There are {} active jobs, at {}".format(list(system_nJobs.values())[0], list(system_nJobs.keys())[0]))

There are 1 active jobs, at 2014-03-31 00:30:02


In [7]:
print(second_key)

2014-03-31 00:36:51


In [8]:
print(third_key)

2014-03-31 00:40:13
